Given a set of elliptic LCS candidates, we seek to extract the closed null geodesic curve corresponding which has the minimum re-intersection distance.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| closed_null_geodesics | list (N,) | list containing $ N $ closed null geodesics
| x_elliptic | list | list containing x-coordinates of elliptic LCS
| y_elliptic | list | list containing y-coordinates of elliptic LCS

In [3]:
# Import numpy
import numpy as np

# Import shapely library to create polygons and points
from shapely.geometry import Polygon, Point

# Import math tools
from math import sqrt

In [4]:
def _elliptic_LCS(closed_null_geodesics):
    
    Poly = []
    return_distance = []
    
    for geodesics in closed_null_geodesics:
        
        if np.sum(geodesics[0]) is not None:
            
            x = geodesics[0]
            y = geodesics[1]
        
            # compute Polygon object of closed null geodesics
            Poly.append(Polygon(zip(x, y)))
            
            # compute re-intersection distance of closed null geodesics
            return_distance.append(sqrt((x[-1]-x[0])**2+(y[-1]-y[0])**2))

    # if no closed null geodesic is found then return 'nan'
    if len(return_distance) == 0:
        return None, None
    
    x_elliptic, y_elliptic = [], []
    
    # if only one closed null geodesic is found --> return it to main function as this is an elliptic OECS.
    if len(return_distance) == 1:
        return x_elliptic.append(x), y_elliptic.append(y)
    
    # sort geodesic curves from minimum re-intersection distance to maximum re-intersection distance:
    return_distance_sorted, closed_null_geodesics_sorted = zip(*sorted(zip(return_distance, closed_null_geodesics)))
    return_distance_sorted, Poly_sorted = zip(*sorted(zip(return_distance, Poly)))

    closed_elliptic_LCS = []
    
    # iterate over all closed curves
    for idx, Poly_geodesics in enumerate(Poly_sorted):
        
        # If first closed elliptic LCS, then set the center as a reference
        if idx == 0:
            
            # coordinates of closed curve
            coords = np.array(Poly_geodesics.exterior.coords)
        
            x = coords[:, 0]
            y = coords[:, 1]
        
            # compute centroid of closed curve
            x_c = np.mean(x)
            y_c = np.mean(y)
            
            closed_elliptic_LCS.append([x_c, y_c, Poly_geodesics])
           
        else:
            
            # assume that closed curve is a new vortex
            bool_create_vortex = True
            
            # iterate over all closed null geodesics
            for v in range(len(closed_elliptic_LCS)):
                
                # If center of elliptic LCS is contained inside the closed curve
                # --> no new elliptic LCS is create.
                if Poly_geodesics.contains(Point(closed_elliptic_LCS[v][0], closed_elliptic_LCS[v][1])):
                    bool_create_vortex = False
                    break
            
            if bool_create_vortex:
                # create new elliptic LCS
                x_c = np.mean(np.array(Poly_geodesics.exterior.coords)[:,0])
                y_c = np.mean(np.array(Poly_geodesics.exterior.coords)[:,1])
                closed_elliptic_LCS.append([x_c, y_c, Poly_geodesics])
    
    # Iterate over all elliptic LCS and store the x/y coordinates.
    for v in closed_elliptic_LCS:
        
        coords = np.array(v[2].exterior.coords)
        
        x = coords[:, 0]
        y = coords[:, 1]
        
        x_elliptic.append(x)
        y_elliptic.append(y)

    return x_elliptic, y_elliptic